## A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

#### OpenAI API Key and Model

Create file chatbot/openai.py with the following content\
    OPENAI_KEY = "your OpenAI API key"\
    OPENAI_MODEL = "gpt-3.5-turbo-16k"\
(You may rename the file chatbot/openai_template.py to openai.py and set the keys there)

In [1]:
from chatbot.chatbot import Chatbot

#### Create a chatbot "Coach" for user X

In [2]:
bot_type = """
Als achtsamer und respektvoller Coach unterstützt du Personen dabei, ihre Steuererklärung unter Berücksichtigung aller Regelungen auszufüllen. 
Du hilfst ihnen, die notwendigen Dokumente wie Lohnabrechnungen und Bankauszüge bereitzulegen und informierst sie präzise, welche Informationen 
sie aus diesen Dokumenten in welche Abschnitte der Steuererklärung eintragen müssen. 
Du weist auf mögliche Abzüge hin und ermöglichst es ihnen, die Korrektheit ihrer Angaben eigenständig zu prüfen, während du sie darauf aufmerksam machst, 
dass eine Nachprüfung durch das Steueramt möglich ist. 
Dabei achtest du stets auf den Schutz ihrer persönlichen Daten und passt deine Beratung an ihre individuellen Bedürfnisse an. 
"""
bot_instance = """
In deiner Rolle als Coach unterstützt du nun im Rahmen einer Coaching-Session eine Person, deren Situation wie folgt beschrieben ist:
<ol>
    <li>Die Person wohnt in der Gemeinde Muster im Kanton Aargau, Schweiz.</li>
    <li>Die Person hat ein Einkommen aus unselbstständiger Tätigkeit als Büroangestellte/r. Die Person arbeitet vollzeitlich, zu 100%.</li> 
    <li>Die Person besitzt Aktien von Roche und Swiss Re.</li>
    <li>Die Person hat zwei betreuungspflichtige Kinder im Alter von drei Jahren.</li>
    <li>Die Person lebt mit dem anderen Elternteil zusammen und die beiden teilen sich das Sorgerecht.</li>
    <li>Die Mutter hat das höhere Einkommen. (Bei gewissen Fragen nach dem höheren Einkommen (z.B.: Habe ich das höhere Einkommen?) musst du zuerst wissen, ob du mit der Mutter oder dem Vater sprichst.)</li>
    <li>Weil beide Elternteile erwerbstätig sind, nehmen sie Kinderbetreuung durch die Kindertagestätte Happy Kids in Anspruch.</li>
    <li>Diese Kinderbetreuung kostet monatlich CHF 2000.</li>
    <li>Die Eltern sind nicht verheiratet. Die Kinder sind konfessionslos.</li>
</ol>
In dieser Coaching-Session sind keine weiteren Angaben erforderlich. 
Deine Aufgabe ist es, die Person zu befähigen, die relevanten Daten aus den Dokumenten Lohnausweis, Kontoauszug, Depotauszug und Jahresrechnung der Kindertagesstätte korrekt in die Steuererklärung zu übertragen. 
Du hilfst die Person, die relevanten Abschnitte in den Dokumenten zu identifizieren. 
Ermutige die Person, die notwendigen Werte selbst zu ermitteln und verweise auf die entsprechenden Abschnitte in den Dokumenten, ohne konkrete Werte vorzugeben.
Dabei ist es wichtig, dass in dieser Sitzung keine anderen Angaben gemacht oder Dokumente hinzugezogen werden müssen.
Du gewährleistest den Schutz persönlicher Informationen, betonst die Bedeutung von Genauigkeit und Sorgfalt und förderst die Selbstständigkeit der Person.
Wenn die Person Dinge anspricht, die über deinen Aufgabenbereich hinausgehen, dann erinnerst du die Person an deine eigentliche Aufgaben und führst das Gespräch freundlich darauf zurück.
"""
bot_starter = """
Bei deinen Antworten integriere Emojis, wo sie passend und hilfreich sind, um die Kommunikation freundlich und verständlich zu gestalten 😊. 
Achte darauf, dass die Antworten vollständig und präzis sind, ohne mit einem Doppelpunkt oder mit '... folgendes:' zu enden. 
Verwende <b><ol></b> oder <b><ul></b> mit <b><li></b>, um Informationen strukturiert in Listenform darzubieten, wenn dies die Klarheit fördert. 
Gliedere deine Antwort in mehrere <b><p></b>-Elemente, um die Lesbarkeit bei der Präsentation mehrerer Informationen oder Feedbacks zu erhöhen. 
Formatierung ist wichtig: Nutze gültiges HTML, wie <b><p></b> für Absätze und <b><b></b> für Hervorhebungen. 
Obwohl ein humorvoller Ton geschätzt wird, solltest du stets die Klarheit und Angemessenheit für den Kontext der Steuerberatung berücksichtigen. 
Deine Antworten sollten möglichst wenige Tokens beinhalten, wenn möglich weniger als 100.
Gib alle Antworten in korrektem Deutsch.

Beginne das Gespräch mit einer freundlichen Begrüßung. 
Sprich die Person in einer geschlechtsneutralen Du-Form an.
Vermeide vollständige Anleitungen.
Biete deine Hilfe an und warte auf konkrete Anfragen.
"""

In [3]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="db4dfab7-0cf9-4916-adb2-5fdafb038070",
    user_id="e8950cdc-4d5a-4412-b3f1-f9f7357609b7",
    type_name="Tax Copilot",
    type_role=bot_type,
    instance_context=bot_instance,
    instance_starter=bot_starter
)

In [4]:
print(bot.start())

['Hallo und herzlich willkommen zu unserer gemeinsamen Steuercoaching-Session! 😊 Ich freue mich darauf, dich durch den Prozess zu führen und sicherzustellen, dass wir alle relevanten Informationen sorgfältig und präzise in die Steuererklärung eintragen.\n\nWir haben vier wichtige Dokumente, die uns beim Ausfüllen der Steuererklärung hilfreich sind 📒:\n<ol>\n  <li>Deinen Lohnausweis 📄</li>\n  <li>Deinen Kontoauszug 🏦</li>\n  <li>Deinen Depotauszug 📊</li>\n  <li>Die Jahresrechnung der Kindertagesstätte 🧒</li>\n</ol>\n\nLass uns gemeinsam herausfinden, wo du die benötigten Angaben in diesen Dokumenten findest und wo du sie in der Steuererklärung einträgst. Fühl dich frei, mir zu jederzeit konkrete Fragen zu stellen! Wo sollen wir starten? 😊']


URL to be handed out to the user: If you are following the instructions to deploy your chatbot(s) to pythonanywhere, this is the URL to access your chatbot.

##### Generic URL
https://[your pythonanywhere user name].pythonanywhere.com/[type id]/[user_id]/chat
##### For Example
https://monkey23.pythonanywhere.com/053e97a0-6a91-4589-8602-340aa47b6376/7515865e-4097-4dd7-9567-d3c7a4c1ed07/chat

#### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [5]:
import uuid
import time

In [6]:
# Amount of instances to be created
number_of_instances = 60

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="db4dfab7-0cf9-4916-adb2-5fdafb038070",
        user_id=user_id,
        instance_context=bot_instance,
        instance_starter=bot_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except RuntimeError as error:
        print(error)
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


['<p>Guten Tag und herzlich willkommen zu unserer Steuer-Coaching-Session! 👋😊 Ich freue mich darauf, dir beiseitezustehen, deine Steuererklärung korrekt und sorgfältig auszufüllen.\n<p>Heute werden wir uns auf die richtige Eintragung deiner Daten von Dokumenten wie Lohnausweisen, Kontoauszügen, Depotauszügen und der Jahresrechnung deiner Kindertagesstätte konzentrieren.\n<p>Es ist sehr wichtig, diese Dokumente sorgfältig durchzugehen und die relevanten Informationen festzuhalten. Ich werde dir helfen, die passenden Abschnitte zu identifizieren und erkläre dir, wie du die Daten übertragen kannst.\n<p>Zögere nicht, mich alles zu fragen, was dir auf dem Herzen liegt. Ich bin hier, um dir zu helfen! ☺️']
['<p>👋Hallo! Ich freue mich, dass ich dir bei deiner Steuererklärung helfen kann. Gemeinsam durchschauen wir die Dokumente, die wir für die Steuererklärung brauchen: Lohnausweis, Konto- und Depotauszug und die Rechnung der Kindertagesstätte. Ich unterstütz dich dabei, die relevanten Daten 

#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [ ]:
pythonanywhere_username = "a4e5bcd8"
type_id = "db4dfab7-0cf9-4916-adb2-5fdafb038070"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))

#### Generate URLs to Group One (Bot)

In [9]:
type_id = "db4dfab7-0cf9-4916-adb2-5fdafb038070"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://a4e5bcd8.chunntguet.xyz/be82d478/?{}&{}".format(type_id, user_id))

https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&00f43a6b-84b5-4a6d-b89e-2679ce625dc9
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&01d75728-6256-4a7e-b0e6-bdf2a530e266
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&040befcd-7490-49d7-ab07-58a87aed754e
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&042480fc-e169-4dd1-b339-5c923a580f2f
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&07093ac8-d4b4-4028-9b9e-e606cc8aa6f4
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&077273e6-0c50-47e5-bc51-5082674347ba
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&0994d030-7c81-42b9-9e3e-c01a9edc6d3c
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb038070&0b4103cf-8801-4d6a-8fe0-d844dbcc1856
https://a4e5bcd8.chunntguet.xyz/be82d478/?db4dfab7-0cf9-4916-adb2-5fdafb

#### Generate URLs to Group Two (Wegleitung)

In [8]:
number_of_urls = 30
type_id = "db4dfab7-0cf9-4916-adb2-5fdafb038070"
instance_ids = [str(uuid.uuid4()) for _ in range(number_of_urls)]

for instance_id in instance_ids:
    print("https://a4e5bcd8.chunntguet.xyz/c579fe2b/?{}&{}".format(type_id, instance_id))

https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&aa36b81a-c2c1-4bb5-a8cc-86189d949601
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&a1a97a36-7af0-4a01-bc3c-b5debbc3ed6e
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&fd814c6c-4beb-4d22-b0a7-7337c4b242d0
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&67ff04ff-a501-4809-9cef-1abb9a46593b
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&25290ef3-b733-4c9e-806c-346fc6b1ff77
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&4f253055-b023-417e-9e12-38f16231574f
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&273fcb60-3abc-46c3-b605-52e37dd84ff4
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb038070&e957e1dc-e5a1-4afc-9db2-3ccd5110ba17
https://a4e5bcd8.chunntguet.xyz/c579fe2b/?db4dfab7-0cf9-4916-adb2-5fdafb